# Tugas Besar Machine Learning Milestone 2

In [1]:
import math
import random
import numpy as np

## Load dataset iris

In [2]:
import pandas as pd
from sklearn import datasets,model_selection,metrics
iris = datasets.load_iris()
iris = pd.DataFrame(iris.data, columns=iris.feature_names)
iris.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
X, y = datasets.load_iris(return_X_y=True)
X_train = X
y_train = y
# print(y_train)
# print(X_train)

## Fungsi utilitas

In [4]:
random.seed(25)

def rand(a, b):
    return (b-a)*random.random() + a

def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

## Fungsi-fungsi aktivasi

In [5]:
def linear(x):
    return x

def dlinear(y):
    return 1.0

def sigmoid(x):
    return math.tanh(x)

def dsigmoid(y):
    return 1.0 - y**2

def relu(x):
    return np.maximum(0.0,x)

def drelu(y):
    if y > 0.0 :
        return 1.0
    else :
        return 0.0

def softmax(x):
    # TODO
    pass

def dsoftmax(y):
    # TODO
    pass

## Kelas Neural Network
note : kode ditulis ulang dari milestone 1 karena representasi bobotnya terlalu susah buat dipake hitung

In [6]:

class NeuralNetwork:
    def __init__(self, ni, nh, no):
        self.ni = ni + 1
        self.nh = nh
        self.no = no

        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no

        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)

        self.activation_function = "sigmoid"

        # init bobot neural network
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.4, 0.4)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-5.0, 5.0)


    def activate(self, inputs):
        assert len(inputs) == self.ni-1, "masukan input salah"

        # aktivasi input layer
        for i in range(self.ni-1):
            self.ai[i] = inputs[i]

        # aktivasi 
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)

        # output activations
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)

        return self.ao[:]


    def backPropagate(self, targets, learning_rate):
        assert len(targets) == self.no, "masukan target salah"

        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        for j in range(self.nh):
            for k in range(self.no):
                w_delta = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + learning_rate*w_delta

        for i in range(self.ni):
            for j in range(self.nh):
                w_delta = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + learning_rate*w_delta

        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error


    def test(self, X_test):
        for x in X_test:
            print(x, '->', self.activate(x))

    def predict(self, X_test):
        prediction = []
        for i in range(len(X_test)):
            prediction.append(self.activate(X_test[i]))
        return prediction

    def createMiniBatches(self, X_train, y_train, batch_size):
        mini_batches = []
        data = np.hstack((X_train, y_train))
        np.random.shuffle(data)
        n_minibatches = data.shape[0] // batch_size

        for i in range(n_minibatches + 1):
            mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
            X_mini = mini_batch[:, :-1]
            Y_mini = mini_batch[:, -1].reshape((-1, 1))
            mini_batches.append((X_mini, Y_mini))
        if data.shape[0] % batch_size != 0:
            mini_batch = data[i * batch_size:data.shape[0]]
            X_mini = mini_batch[:, :-1]
            Y_mini = mini_batch[:, -1].reshape((-1, 1))
            mini_batches.append((X_mini, Y_mini))
        return mini_batches

    def train(self, X_train, y_train, iterations=30, learning_rate=0.01, error_treshold = 0.01, activation_function="sigmoid", batch_size = 10):
        # N: learning rate
        self.activation_function = activation_function
        for i in range(0, iterations):
            error = 0.0
            mini_batches = self.createMiniBatches(X_train, y_train, batch_size)
            for mini_batch in mini_batches:
                error = 0.0
                for j in range(0, len(X_train)):
                    inputs = X_train[j]
                    targets = y_train[j]
                    self.activate(inputs)
                    error = error + self.backPropagate(targets, learning_rate)

                if i % 10 == 0:
                    print('error %-.5f' % error)

                if error < error_treshold:
                    print('final error %-.5f' % error)
                    return


In [7]:
X_dataset, y_dataset = datasets.load_iris(return_X_y=True)
X_train = X_dataset
y_train = []
for y in y_dataset:
    y_train.append([y])
print(y_train)
iris_nn = NeuralNetwork(len(X_train[0]), 10, 1) # Mendefinisikan ukuran neural network
iris_nn.train(X_train,y_train, activation_function="sigmoid", error_treshold = 10)
iris_nn.test(X_train)

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]
error 49.99939
error 49.99938
error 49.99937
error 49.99935
error 49.99934
error 49.99932
error 49.99930
error 49.99928
error 49.99927
error 49.99925
error 49.99923
error 49.99920
error 49.99918
error 49.99915
error 49.99913
error 49.99910
error 25.